In [141]:
import random
import copy

class Team:
  def __init__(self, name, wins, games, points):
    self._name = name
    self._points = points
    self._wins = wins
    self._games = games
    if(wins != 0 and games != 0):
        self._win_rate = wins/games
    else:
        self._win_rate = 0
        
  def update_win_rate(self):
    self._win_rate = self._wins/self._games    
    
  def win(self):   
    self._games = self._games + 1
    self._wins = self._wins + 1
    self.update_win_rate()  
    
  def lose(self):
    self._games = self._games + 1 
    self.update_win_rate() 
    
  def tie(self):
    self._games = self._games + 1 
    self.update_win_rate()    
    
  def win_points(self):
    self._points = self._points + 3
    self._games = self._games + 1
    self._wins = self._wins + 1
    self.update_win_rate()  
    
  def lose_points(self):
    self._games = self._games + 1 
    self.update_win_rate() 
    
  def tie_points(self):
    self._points = self._points + 1
    self._games = self._games + 1 
    self.update_win_rate()    
    
  def __repr__(self):
    return (self._name +" : " + str(self._points) + " / "+ str(self._wins)+ " / "+ str(self._games)+ " / "+ str(self._win_rate) + "\n")


def search_team_one(gameString,teamList,team_one):
  ele = gameString.split(",")
  teams = [ele[4],ele[5]]
  if(teams[1][0] == ' '):
    teams[1] = teams[1][1:]
  if(teams[1][-1] == '\n'):
    teams[1] = teams[1][:-1]
  for group in teamList:
    for team in group:   
        if(team._name == teams[0]):      
            return team

def search_team_two(gameString,teamList,team_two):
  ele = gameString.split(",")
  teams = [ele[4],ele[5]]
  if(teams[1][0] == ' '):
    teams[1] = teams[1][1:]
  if(teams[1][-1] == '\n'):
    teams[1] = teams[1][:-1]
  for group in teamList:
    for team in group:   
        if(team._name == teams[1]):      
            return team

In [142]:
def simulate_game(team_one,team_two,current_team_list):
  status = "UNDEFINED"
  rng = random.uniform(0,1)
  rng_two = random.uniform(0,1)

  if(team_one._win_rate >= rng):
    if(team_two._win_rate < rng_two):
      team_one.win_points()
      team_two.lose_points()
      status = "ONE WON"   
    else:
      team_one.tie_points()
      team_two.tie_points()
      status = "TIE"
  else:
    if(team_two._win_rate >= rng_two):
      team_two.win_points()
      team_one.lose_points()
      status = "TWO WON"   
    else:
      team_one.tie_points()
      team_two.tie_points()
      status = "TIE"

def simulate_game_elm(team_one,team_two,current_team_list):
  status = "UNDEFINED"

  while(status == "UNDEFINED" or status == "TIE"):
        rng = random.uniform(0,1)
        rng_two = random.uniform(0,1)  
        
        if(team_one._win_rate >= rng):
            if(team_two._win_rate < rng_two):      
              team_one.win()
              current_team_list.pop(current_team_list.index(team_two)) 
              status = "WIN"
            else:     
              status = "TIE"
        else:
            if(team_two._win_rate >= rng_two):      
              team_two.win()
              current_team_list.pop(current_team_list.index(team_one)) 
              status = "WIN"
            else:      
              status = "TIE"
            
def initialize_list(original_team_list):
    base_name = "group"
    original_team_list = [[],[],[],[],[],[],[],[]]
    for j in range (0,8):
        current_file = base_name + str(j)+ ".txt"
        f = open(current_file, "r",encoding="utf-8-sig")        
        for line in f.readlines():
            ele = line.split(";")
            if(ele[0] != '\n'):
                ele[3] = ele[3][:-1]         
                temp_team = Team(ele[0],int(ele[2]),int(ele[3]),int(ele[1]))
                original_team_list[j].append(temp_team)
    return original_team_list

def ordenate_list_groups(current_team_list):
    for group in current_team_list:
        group.sort(key=lambda x: x._points, reverse=True)  
    return current_team_list

def extract_winners_groups(current_team_list):
    winner_list = []
    for group in current_team_list:
        winner_list.append(group[0])
        winner_list.append(group[1])
    return winner_list

In [190]:
%%time

k = 10000
current_team_list = [[],[],[],[],[],[],[],[]]
times_brazil_s_p = 0 
second_phase_teams = []
second_phase_frames = []
brazil_group_frames = []
            
for i in range(0,k):   
  current_team_list = initialize_list(current_team_list)
  t = open("jogos_copa.txt", "r",encoding="utf-8-sig")
  team_one = Team("",0,0,0)
  team_two = Team("",0,0,0)          

  for game in t.readlines():  
    team_one = search_team_one(game,current_team_list,team_one)
    team_two = search_team_two(game,current_team_list,team_two)  
    simulate_game(team_one,team_two,current_team_list)
    
  current_team_list = ordenate_list_groups(current_team_list)
  second_phase_teams = extract_winners_groups(current_team_list)
    
  if(current_team_list[6][0]._name == "Brasil" or current_team_list[6][1]._name == "Brasil"):
    times_brazil_s_p = times_brazil_s_p + 1
    
  found = False
  current_group_string = ""
  for team in current_team_list[6]:
    current_group_string = current_group_string + team._name + ":" + str(team._points) + ";"
  for groupAndCounter in brazil_group_frames:     
    if(groupAndCounter[0] == current_group_string):
      found = True
      groupAndCounter[1] = groupAndCounter[1] + 1
  if(not found):
    brazil_group_frames.append([current_group_string,1]) 
    
  found_win_list =  False
  current_win_list = ""
  for team in second_phase_teams:
    current_win_list = current_win_list + team._name + ";"
  for groupAndCounter in second_phase_frames:     
    if(groupAndCounter[0] == current_win_list):
      found_win_list = True
      groupAndCounter[1] = groupAndCounter[1] + 1
  if(not found_win_list):
    second_phase_frames.append([current_win_list,1])

brazil_group_frames.sort(key=lambda x: x[1], reverse=True)   
for group in brazil_group_frames:
    group[1] = (group[1] / k) * 100

second_phase_frames.sort(key=lambda x: x[1], reverse=True)   
for group in second_phase_frames:
    group[1] = (group[1] / k) * 100    

CPU times: user 11.5 s, sys: 499 ms, total: 12 s
Wall time: 12 s


In [194]:
print("->Porcentagem de vezes que o Brasil classificou para 2 fase: "+ str(times_brazil_s_p/k *100)+"%.")
print("->Quadros e porcentagens do grupo do Brazil:")
for frame in brazil_group_frames[:10]:
    print(frame)
print("->Quadros e porcentagens da segunda fase:")
for frame in second_phase_frames[:10]:
    print(frame)    
  

->Porcentagem de vezes que o Brasil classificou para 2 fase: 85.53%.
->Quadros e porcentagens do grupo do Brazil:
['Brasil:7;Sérvia:3;Suíça:2;Camarões:2;', 1.87]
['Brasil:5;Sérvia:3;Suíça:3;Camarões:2;', 1.7999999999999998]
['Brasil:3;Sérvia:3;Suíça:3;Camarões:3;', 1.66]
['Brasil:5;Sérvia:5;Suíça:2;Camarões:2;', 1.58]
['Brasil:5;Sérvia:3;Camarões:3;Suíça:2;', 1.54]
['Brasil:9;Sérvia:2;Suíça:2;Camarões:2;', 1.53]
['Brasil:7;Suíça:3;Sérvia:2;Camarões:2;', 1.4000000000000001]
['Brasil:7;Sérvia:4;Suíça:2;Camarões:2;', 1.37]
['Brasil:5;Suíça:3;Camarões:3;Sérvia:2;', 1.27]
['Brasil:7;Camarões:3;Sérvia:2;Suíça:2;', 1.22]
->Quadros e porcentagens da segunda fase:
['Equador;Holanda;Inglaterra;Irã;Arábia Saudita;México;Austrália;França;Alemanha;Espanha;Marrocos;Bélgica;Brasil;Sérvia;Portugal;Gana;', 0.02]
['Holanda;Senegal;Irã;Inglaterra;Arábia Saudita;Polônia;Tunísia;Dinamarca;Costa Rica;Alemanha;Croácia;Bélgica;Sérvia;Camarões;Gana;Uruguai;', 0.01]
['Catar;Holanda;Estados Unidos;Inglaterra;Ará

In [195]:
 print(second_phase_teams)
coup_winners = []
for i in range(0,k_simulations):    
    second_phase_teams_current = second_phase_teams.copy()
    
    zeroth_team = second_phase_teams_current[0]
    first_team = second_phase_teams_current[1]
    second_team = second_phase_teams_current[2]
    third_team = second_phase_teams_current[3]
    fourth_team = second_phase_teams_current[4]
    fifth_team = second_phase_teams_current[5]
    sixth_team = second_phase_teams_current[6]
    seventh_team = second_phase_teams_current[7]
    
    eight_team = second_phase_teams_current[8]
    nineth_team = second_phase_teams_current[9]
    tenth_team = second_phase_teams_current[10]
    eleventh_team = second_phase_teams_current[11]
    twelveth_team = second_phase_teams_current[12]
    thirteenth_team = second_phase_teams_current[13]
    fourteenth_team = second_phase_teams_current[14]
    fifteenth_team = second_phase_teams_current[15]
    

    second_phase_teams_current = simulate_game_elm(zeroth_team,third_team,second_phase_teams_current)  
    second_phase_teams_current = simulate_game_elm(first_team,second_team,second_phase_teams_current) 
    second_phase_teams_current = simulate_game_elm(fourth_team,seventh_team,second_phase_teams_current)   
    second_phase_teams_current = simulate_game_elm(fifth_team,sixth_team,second_phase_teams_current)    
    second_phase_teams_current = simulate_game_elm(eight_team,eleventh_team,second_phase_teams_current)  
    second_phase_teams_current = simulate_game_elm(nineth_team,tenth_team,second_phase_teams_current) 
    second_phase_teams_current = simulate_game_elm(twelveth_team,fifteenth_team,second_phase_teams_current)   
    second_phase_teams_current = simulate_game_elm(thirteenth_team,fourteenth_team,second_phase_teams_current) 
    
    zeroth_team = second_phase_teams_current[0]
    first_team = second_phase_teams_current[1]
    second_team = second_phase_teams_current[2]
    third_team = second_phase_teams_current[3]
    fourth_team = second_phase_teams_current[4]
    fifth_team = second_phase_teams_current[5]
    sixth_team = second_phase_teams_current[6]
    seventh_team = second_phase_teams_current[7]
    
    second_phase_teams_current = simulate_game_elm(zeroth_team,third_team,second_phase_teams_current)  
    second_phase_teams_current = simulate_game_elm(first_team,second_team,second_phase_teams_current) 
    second_phase_teams_current = simulate_game_elm(fourth_team,seventh_team,second_phase_teams_current)   
    second_phase_teams_current = simulate_game_elm(fifth_team,sixth_team,second_phase_teams_current)
    
    zeroth_team = second_phase_teams_current[0]
    first_team = second_phase_teams_current[1]
    second_team = second_phase_teams_current[2]
    third_team = second_phase_teams_current[3]
    
    second_phase_teams_current = simulate_game_elm(zeroth_team,third_team,second_phase_teams_current)  
    second_phase_teams_current = simulate_game_elm(first_team,second_team,second_phase_teams_current) 
    
    zeroth_team = second_phase_teams_current[0]
    first_team = second_phase_teams_current[1]
    
    second_phase_teams_current = simulate_game_elm(zeroth_team,first_team,second_phase_teams_current) 
    
    found_winner = False
    for teamAndCounter in coup_winners:     
        if(teamAndCounter[0] == second_phase_teams_current[0]):
          found_winner = True
          teamAndCounter[1] = teamAndCounter[1] + 1
    if(not found_winner):
        coup_winners.append([second_phase_teams_current[0],1])  

coup_winners.sort(key=lambda x: x[1], reverse=True)   
for group in coup_winners:
    group[1] = group[1] / k_simulations
    print(group)    
    


[Equador : 7 / 92 / 258 / 0.35658914728682173
, Holanda : 4 / 192 / 460 / 0.41739130434782606
, Inglaterra : 5 / 234 / 429 / 0.5454545454545454
, Irã : 3 / 92 / 198 / 0.46464646464646464
, Arábia Saudita : 7 / 99 / 243 / 0.4074074074074074
, México : 5 / 248 / 521 / 0.4760076775431862
, França : 5 / 224 / 508 / 0.4409448818897638
, Austrália : 3 / 144 / 314 / 0.4585987261146497
, Japão : 7 / 106 / 253 / 0.4189723320158103
, Alemanha : 6 / 311 / 589 / 0.5280135823429541
, Croácia : 9 / 74 / 151 / 0.4900662251655629
, Bélgica : 4 / 173 / 467 / 0.37044967880085655
, Suíça : 5 / 13 / 40 / 0.325
, Brasil : 3 / 73 / 112 / 0.6517857142857143
, Portugal : 7 / 115 / 289 / 0.39792387543252594
, Gana : 4 / 79 / 224 / 0.35267857142857145
]


AttributeError: 'NoneType' object has no attribute 'pop'